In [ ]:
import talib
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import yfinance as yf
import plotly.express as px
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta, date
import backtesting
from backtesting import Backtest, Strategy
import warnings
import itertools
import optuna

Выбираем 9 финансовых инструментов для анализа: "AMZN", "GOOG", "TSLA", "MBG.DE", "DTG.DE", "VOW.DE", "BTC-USD", "ETH-USD", "SOL-USD" (три американские акции, три немецкие и три криптовалюты).  
Интервал - 1 день, период - с 01 февраля 2022 до вчерашнего дня.  
Скачиваем данные с YAHOO и складываем их в словарь df_dict.  
В дальнейшем можно сделать словарь словарей для хранения технических индикаторов и других данный по инструменту в 
одном словаре.

In [39]:
df_dict = {}
tickers = ["AMZN", "GOOG", "TSLA", "MBG.DE", "DTG.DE", "VOW.DE", "BTC-USD", "ETH-USD", "SOL-USD"]
yesterday = date.today() - timedelta(days = 1)

for ticker in tickers:

    df_dict[ticker] = yf.download(ticker, start='2022-02-01', end=yesterday, interval='1d').drop(columns=['Adj Close'])
    df_dict[ticker].columns = [f'{Price}' for Price, Ticker in df_dict[ticker].columns]
    df_dict[ticker] = df_dict[ticker].reset_index()
    df_dict[ticker].columns = df_dict[ticker].columns.str.lower()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [40]:
class TemaMacdStrategy(Strategy):
    
    def init(self):
        self.signal = self.I(lambda: self.data.Signal)
        self.previous_signal = 0
        self.size = 0.1

    def next(self):
        current_signal = self.signal[-1]

        if current_signal != self.previous_signal:
            if current_signal == 1:
                if self.position.is_short:
                    self.position.close()
                    
                if not self.position.is_long:
                    self.buy(size=self.size)
                    
            elif current_signal == -1:
                if self.position.is_long:
                    self.position.close()
                   
                if not self.position.is_short:
                    self.sell(size=self.size)
                    
            elif current_signal == 0:
                if self.position:
                    self.position.close()

        self.previous_signal = current_signal

### Walk Forward Optimization

In [41]:
# напишем функцию стартегии
def apply_strategy(data, params):
    """
    Применение стратегии с оптимизированными параметрами.

    :param df: DataFrame с данными, на которые будут наложены индикаторы.
    :param params: Словарь с оптимизированными параметрами.
    :return: DataFrame с рассчитанными индикаторами и сигналами.
    """
    df = data.copy()
    # Извлекаем параметры из словаря
    rsi_period = params['rsi_period']
    fastMACD_period = params['fastMACD_period']
    slowMACD_period = params['slowMACD_period']
    signalMACD_period = params['signalMACD_period']
    
    # Добавляем индикаторы
    df['macd'], df['macd_signal'], df['macd_hist'] = talib.MACD(df['close'], fastperiod=fastMACD_period, slowperiod=slowMACD_period, signalperiod=signalMACD_period)
    df['rsi'] = talib.RSI(df['close'], timeperiod=rsi_period)

    # Создаем сигналы для покупки и продажи
    df['signal'] = 0
    # df.loc[(df['macd'] > df['macd_signal']) & (50 <= df['rsi']) & (df['rsi'] < 80), 'signal'] = 1  # Сигнал на покупку
    # df.loc[(df['macd'] < df['macd_signal']) & (20 < df['rsi']) & (df['rsi'] < 50), 'signal'] = -1  # Сигнал на продажу
    df.loc[(df['macd'] > df['macd_signal']) & (((50 <= df['rsi']) & (df['rsi'] < 70)) | ((0 <= df['rsi']) & (df['rsi'] < 30))), 'signal'] = 1  # Сигнал на покупку
    df.loc[(df['macd'] < df['macd_signal']) & (((30 <= df['rsi']) & (df['rsi'] < 50)) | ((70 <= df['rsi']) & (df['rsi'] < 100))), 'signal'] = -1  # Сигнал на продажу
    
    return df[["date", "open", "high", "low", "close", "volume","signal"]]

def bactest_strategy(df, strategy_class, params, plot=False):
    """
    Запускает бэктест с переданными параметрами стратегии.

    :param df: DataFrame с данными для бэктеста.
    :param strategy_class: Класс стратегии для бэктеста.
    :param params: Словарь с параметрами стратегии.
    :return: Статистика бэктеста.
    """
    # Применяем стратегию с переданными параметрами
    
    df = apply_strategy(df.copy(), params)
    
    # Подготовка данных для бэктеста
    bt_df = df.copy()
    bt_df.columns = bt_df.columns.str.capitalize()
    bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
    bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
    bt_df.set_index('Datetime', inplace=True)

    # Создаем объект класса Backtest с текущей стратегией
    bt = Backtest(bt_df, strategy_class, cash=500000, commission=.002, exclusive_orders=True, margin=0.1)

    # Запускаем бэктест
    stats = bt.run()
    if plot:
        bt.plot(
        plot_equity=True,
        plot_drawdown=True,
        relative_equity=False,
        )
    return stats

In [66]:
def get_best_strategy(buffer, strategy_class):

# OPTUNA

    def objective(trial):


        rsi_period = trial.suggest_int('rsi_period', 12, 15)
        fastMACD_period = trial.suggest_int('fastMACD_period', 12, 15)
        slowMACD_period = trial.suggest_int('slowMACD_period', 21, 38)
        signalMACD_period = trial.suggest_int('signalMACD_period', 7, 10)

        # Создаем словарь с текущими параметрами
        params = {
            'rsi_period': rsi_period,
            'fastMACD_period': fastMACD_period,
            'slowMACD_period': slowMACD_period,
            'signalMACD_period': signalMACD_period
        }

        # Запускаем бэктест с текущими параметрами
        stats = bactest_strategy(buffer.copy(), strategy_class, params)
        # Оптимизируем по Sharpe Ratio
        return stats['Sharpe Ratio']
        #return stats['Profit Factor']
        

    study = optuna.create_study(direction='maximize')
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study.optimize(objective, n_trials=30)

    #print(study.best_params)
    return study.best_params


In [ ]:
btest_res = pd.DataFrame()
for ticker in tickers:

    df = df_dict[ticker].copy()
    train_size = 90  # Размер окна тренировки
    test_size = 30    # Размер тестового окна

    # Инициализация DataFrame для сигналов
    signals_df = pd.DataFrame()

    # Определение количества итераций
    num_iterations = (len(df) - train_size) // test_size

    for i in range(num_iterations + 1):
        # Определение границ обучающего и тестового окон
        start_train = i * test_size
        end_train = start_train + train_size
        start_test = end_train
        end_test = start_test + test_size
        
        # Если конец тестового окна выходит за пределы данных, обрезаем его
        if end_test > len(df):
            end_test = len(df)
        
        # Определяем окна для тренировки и тестирования
        train_data = df.iloc[start_train:end_train].copy()
        test_data = df.iloc[start_test:end_test].copy()
        
        
        # Оптимизация на тренировочном окне
        best_params = get_best_strategy(train_data, TemaMacdStrategy)
        
        # Объединяем данные тренировки и теста
        combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)
        
        # Применяем стратегию на объединенном окне с оптимальными параметрами
        combined_with_signal = apply_strategy(combined_data.copy(), best_params)
        
        # Извлекаем только часть данных, относящуюся к тестовому окну
        test_with_signal = combined_with_signal.iloc[-test_size:].copy()
        
        # Добавляем сигналы из тестового окна в signals_df
        signals_df = pd.concat([signals_df, test_with_signal], ignore_index=True)

    bt_df = signals_df.copy()
    bt_df.columns = bt_df.columns.str.capitalize()
    bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
    bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
    bt_df.set_index('Datetime', inplace=True)
    bt_df = bt_df.sort_index()

    bt = Backtest(bt_df, TemaMacdStrategy, cash=500000, commission=0.002, exclusive_orders=True, margin=0.1)

        # Запускаем бэктест
    stats = bt.run()
    btest_res[ticker] = stats[:25]
    # print(ticker)
    # print(stats[:25])


In [70]:
btest_res

,AMZN,GOOG,TSLA,MBG.DE,DTG.DE,VOW.DE,BTC-USD,ETH-USD,SOL-USD
Start,2022-06-10 00:00:00+00:00,2022-06-10 00:00:00+00:00,2022-06-10 00:00:00+00:00,2022-06-09 00:00:00+00:00,2022-06-09 00:00:00+00:00,2022-06-09 00:00:00+00:00,2022-05-02 00:00:00+00:00,2022-05-02 00:00:00+00:00,2022-05-02 00:00:00+00:00
End,2024-12-13 00:00:00+00:00,2024-12-13 00:00:00+00:00,2024-12-13 00:00:00+00:00,2024-12-13 00:00:00+00:00,2024-12-13 00:00:00+00:00,2024-12-13 00:00:00+00:00,2024-12-14 00:00:00+00:00,2024-12-14 00:00:00+00:00,2024-12-14 00:00:00+00:00
Duration,917 days 00:00:00,917 days 00:00:00,917 days 00:00:00,918 days 00:00:00,918 days 00:00:00,918 days 00:00:00,957 days 00:00:00,957 days 00:00:00,957 days 00:00:00
Exposure Time [%],72.03125,75.15625,66.5625,66.153846,68.615385,68.615385,61.145833,68.645833,64.0625
Equity Final [$],541019.356192,351531.91684,1603399.984655,554443.213215,565382.070517,254317.917032,591258.995156,585666.671796,1091806.880167
Equity Peak [$],614149.470913,500000.0,1653526.332799,599717.662839,701535.388966,558580.102911,745842.468723,1128522.295899,1207268.112921
Return [%],8.203871,-29.693617,220.679997,10.888643,13.076414,-49.136417,18.251799,17.133334,118.361376
Buy & Hold Return [%],107.441864,71.752941,87.843956,-15.563852,26.604251,-56.765125,163.105986,35.381507,151.014167
Return (Ann.) [%],3.19384,-13.105642,59.143224,4.10775,4.90299,-23.149188,6.595754,6.210491,34.656306
Volatility (Ann.) [%],29.392174,22.322297,71.932406,21.950586,22.395902,18.704148,40.181356,52.620096,98.798695


Нарисуем для сравнения все 9 графиков отношения цены закрытия к средней цене закрытия за наш период наблюдений. 
Видим рост волатильности с разной скоростью у разных инструментов с февраля 2024.

In [45]:
for ticker in tickers:

    df_dict[ticker].index = pd.to_datetime(df_dict[ticker].date)

In [46]:
fig = go.Figure()

# Добавляем графики
for ticker in tickers:
    price_mean = df_dict[ticker].close.mean()
    fig.add_trace(go.Scatter(x=df_dict[ticker].index, y=(df_dict[ticker].close)/price_mean,                         
                             mode='lines', name=ticker,
                             line=dict(width=1)))

# Настройка осей и названий
fig.update_layout(title='Interval volatility',
                  xaxis_title='Date',
                  yaxis_title='Close Price/price_mean',
                  template='plotly_dark')

# Отображаем график
fig.show()